In [9]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [10]:
#1 Import Required Libraries and Setup Environment

import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import MobileNetV2
import numpy as np

In [ ]:
#2 Load & Unzip own Dataset

!unzip /content/dataset.zip

In [11]:
#3 Provide Path for Training and Validation

train_dir = '/content/dataset/train'
validation_dir = '/content/dataset/validation'

In [12]:
#4 Preprocess Images & Perform Data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

Found 1213 images belonging to 8 classes.
Found 585 images belonging to 8 classes.


In [13]:
#5 Load pre-trained MobileNetV2 model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(150, 150, 3))


In [14]:
#6 Add global average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

In [15]:
#7 Model to train
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze all layers of the base model
for layer in base_model.layers:
    layer.trainable = False

In [16]:
#8 Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display model architecture
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 150, 150, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 75, 75, 32)           864       ['input_2[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 75, 75, 32)           128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 75, 75, 32)           0         ['bn_Conv1[0][0]']      

In [17]:
#9 Train the model
history = model.fit(
    train_generator,
    epochs=15,
    validation_data=validation_generator
)

Epoch 1/15
38/38 [==============================] - 13s 225ms/step - loss: 2.7631 - accuracy: 0.2201 - val_loss: 2.9328 - val_accuracy: 0.2256
Epoch 2/15
38/38 [==============================] - 9s 241ms/step - loss: 2.1816 - accuracy: 0.3075 - val_loss: 2.5336 - val_accuracy: 0.3179
Epoch 3/15
38/38 [==============================] - 9s 243ms/step - loss: 2.0028 - accuracy: 0.3298 - val_loss: 2.1165 - val_accuracy: 0.3402
Epoch 4/15
38/38 [==============================] - 8s 224ms/step - loss: 1.9128 - accuracy: 0.3660 - val_loss: 2.0920 - val_accuracy: 0.3282
Epoch 5/15
38/38 [==============================] - 9s 220ms/step - loss: 1.7653 - accuracy: 0.3875 - val_loss: 2.1058 - val_accuracy: 0.3282
Epoch 6/15
38/38 [==============================] - 9s 245ms/step - loss: 1.7394 - accuracy: 0.3784 - val_loss: 2.0470 - val_accuracy: 0.3521
Epoch 7/15
38/38 [==============================] - 8s 217ms/step - loss: 1.6583 - accuracy: 0.4295 - val_loss: 1.9296 - val_accuracy: 0.3709
Epoch

In [18]:
#10 Evaluate the model
loss, accuracy = model.evaluate(validation_generator)
print(f'Validation loss: {loss}')
print(f'Validation accuracy: {accuracy}')

# Save the model
model.save('face_recognition_model_mobilenetv2.h5')

19/19 [==============================] - 1s 36ms/step - loss: 1.8489 - accuracy: 0.3675
Validation loss: 1.848912239074707
Validation accuracy: 0.36752137541770935


In [19]:
#11 Prediction functions
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(150, 150))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Rescale to match the training rescale
    return img_array

In [20]:
def predict_image_class(img_path, model, class_indices):
    img_array = preprocess_image(img_path)
    predictions = model.predict(img_array)
    predicted_class_index = np.argmax(predictions, axis=1)
    predicted_class = list(class_indices.keys())[list(class_indices.values()).index(predicted_class_index)]
    return predicted_class

In [21]:
#12 Load the model for prediction
model = load_model('face_recognition_model_mobilenetv2.h5')

#13 Get class indices from the training generator
class_indices = train_generator.class_indices

In [23]:
#14 Make a prediction

img_path = '/content/klrahul.jpg'
predicted_class = predict_image_class(img_path, model, class_indices)
print(f'Predicted class: {predicted_class}')

1/1 [==============================] - 0s 24ms/step
Predicted class: MSDhoni
